In [1]:
import pandas as pd
#import nltk and sklearn
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


In [2]:
# complaints= pd.read_csv('complaint with manual labels.csv')
complaints= pd.read_csv('shuffled data.csv')
test= pd.read_csv('shuffled test.csv')
complaints


,hindi,complaint,category
0,गांव भालौठ (रोहतक) के पास एक कार को ट्रक ने पी...,NaN,accident
1,व्यक्ति ने बताया वह साइकिल पर घर जा रहा था कि ...,NaN,accident
2,पंचकूला में कर्जे से परेशान हो क्र एक आदमी ने ...,NaN,theft
3,बच्ची की मां ने बताया कि उसकी बेटा के एक सहपाठ...,NaN,rape
4,"पंचकुला का रहना वाला आरोपी पवन कुमार, (जो वर्त...",NaN,theft
...,...,...,...
347,गांव खरकडा (हांसी) से एक क्रूजर गाड़ी जिसमें 6...,NaN,accident
348,शनिवार देर शाम को सेक्टर 14 में रहने वाले बजे ...,NaN,murder
349,मुकेश वासी चरखी दादरी अपनी पत्नी व बेटे के साथ...,NaN,accident
350,लड़की के परिवारवालों का कहना है कि आरोपी की भाभ...,NaN,rape


In [3]:
translated_data= pd.read_csv("translated data.csv")
translated_test = pd.read_csv("translated test.csv")


complaints['complaint'] = translated_data['complaint']
test['complaint'] = translated_test['complaint']

In [4]:
complaints.head()

,hindi,complaint,category
0,गांव भालौठ (रोहतक) के पास एक कार को ट्रक ने पी...,A car was hit from behind by a truck near vill...,accident
1,व्यक्ति ने बताया वह साइकिल पर घर जा रहा था कि ...,The person told that he was going home on a bi...,accident
2,पंचकूला में कर्जे से परेशान हो क्र एक आदमी ने ...,"Troubled by debt, a man along with his friend ...",theft
3,बच्ची की मां ने बताया कि उसकी बेटा के एक सहपाठ...,The girl's mother told that a classmate of her...,rape
4,"पंचकुला का रहना वाला आरोपी पवन कुमार, (जो वर्त...","Accused Pawan Kumar, a resident of Panchkula, ...",theft


In [7]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Garima\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Garima\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
import re
import nltk

# We need this dataset in order to use the tokenizer
# nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Also download the list of stopwords to filter out
# nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def process_text(text):
    # Make all the strings lowercase and remove non alphabetic characters
    text = re.sub('[^A-Za-z]', ' ', text.lower())

    # Tokenize the text; this is, separate every sentence into a list of words
    # Since the text is already split into sentences you don't have to call sent_tokenize
    tokenized_text = word_tokenize(text)

    # Remove the stopwords and stem each word to its root
    clean_text = [
        stemmer.stem(word) for word in tokenized_text
        if word not in stopwords.words('english')
    ]

    # Remember, this final output is a list of words
    return clean_text

In [6]:
texts = complaints['complaint']
topics = complaints['category']

# Process the texts to so they are ready for training
# But transform the list of words back to string format to feed it to sklearn
texts = [" ".join(process_text(text)) for text in texts]

In [7]:
texts

['car hit behind truck near villag bhalauth rohtak peopl car die spot',
 'person told go home bicycl white car came behind hit fell got injur',
 'troubl debt man along friend loot girlfriend hous panchkula abscond robberi',
 'girl mother told classmat son tri rape lunch break school matter sirsa',
 'accus pawan kumar resid panchkula present resid rohini delhi live like lavish lifestyl two girlfriend one delhi nativ place one sushil agarw march complaint receiv given famili driver pawan kumar netaji subhash place drop amount rs lakh nephew offic way said money loot car near hindu rao hospit pawan kumar bmw tire chang driver question felt complic incid',
 'kmp e kundli manesar palwal expressway pass bahadurgarh seen havoc speed two peopl die two separ accid soon incid report polic reach spot bodi brought gener hospit bahadurgarh postmortem ident person kill accid yet ascertain',
 'car hit kavadiya near villag beed sotti kurukshetra brought lnjp hospit kurukshetra injur condit treatment w

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer(max_features=1000)
vectors = matrix.fit_transform(texts).toarray()

In [9]:
print(vectors)
print(vectors.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(352, 1000)


In [10]:
from sklearn.model_selection import train_test_split
vectors_train, vectors_val, topics_train, topics_val = train_test_split(vectors, topics, test_size=0.4)

In [11]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(vectors_train, topics_train)

GaussianNB()

In [12]:
# Predict with the testing set
topics_pred = classifier.predict(vectors_val)

# ...and measure the accuracy of the results
from sklearn.metrics import classification_report
print(classification_report(topics_val, topics_pred))

              precision    recall  f1-score   support

    accident       0.89      0.81      0.85        31
      murder       0.54      0.66      0.59        32
        rape       0.82      0.79      0.80        42
       theft       0.85      0.81      0.83        36

    accuracy                           0.77       141
   macro avg       0.78      0.76      0.77       141
weighted avg       0.78      0.77      0.77       141



In [13]:
newcomplaints = [
    "I was a victim of theft. My wallet got stolen.",
    "I witnessed a car accident on Main Street.",
    "My sister has been killed .",
    "my package worth 50$ was stolen.",
    "There has been a murder in my neighborhood. Please investigate.",
    "I suspect my coworker of stealing office supplies.",
]

newvectors = matrix.transform(newcomplaints).toarray()

In [14]:
predictions= [classifier.predict(newvectors)]
predictions

[array(['theft', 'theft', 'rape', 'theft', 'rape', 'rape'], dtype='<U8')]

In [17]:
#test on test data
# test= pd.read_csv('testdata.csv')
complaints= test['complaint']
newvectors = matrix.transform(complaints).toarray()
predictions= classifier.predict(newvectors)
# predictions= predictions.reshape(-1,1)
# predictions

In [20]:
print(classification_report(test['category'], predictions))
print(accuracy_score(test['category'], predictions))

              precision    recall  f1-score   support

    accident       0.58      1.00      0.73        11
      murder       0.70      0.70      0.70        10
        rape       0.56      0.56      0.56         9
       theft       1.00      0.53      0.69        17

    accuracy                           0.68        47
   macro avg       0.71      0.70      0.67        47
weighted avg       0.75      0.68      0.68        47

0.6808510638297872


In [21]:
from sklearn.svm import SVC
classifier = SVC()
classifier.fit(vectors_train, topics_train)

SVC()

In [22]:
# Predict with the testing set
topics_pred = classifier.predict(vectors_val)

# ...and measure the accuracy of the results
from sklearn.metrics import classification_report
print(classification_report(topics_val, topics_pred))

              precision    recall  f1-score   support

    accident       0.93      0.87      0.90        31
      murder       0.63      0.81      0.71        32
        rape       0.89      0.81      0.85        42
       theft       0.91      0.83      0.87        36

    accuracy                           0.83       141
   macro avg       0.84      0.83      0.83       141
weighted avg       0.85      0.83      0.83       141



In [23]:
newcomplaints = [
    "I was a victim of theft. My wallet got stolen.",
    "I witnessed a car accident on Main Street.",
    "My sister has been killed .",
    "my package worth 50$ was stolen.",
    "There has been a murder in my neighborhood. Please investigate.",
    "I suspect my coworker of stealing office supplies.",
]

newvectors = matrix.transform(newcomplaints).toarray()

In [24]:
predictions= [classifier.predict(newvectors)]
predictions

[array(['theft', 'murder', 'murder', 'theft', 'murder', 'murder'],
       dtype=object)]

In [25]:
#test on test data
# test= pd.read_csv('testdata.csv')
complaints= test['complaint']
newvectors = matrix.transform(complaints).toarray()
predictions= classifier.predict(newvectors)
# predictions= predictions.reshape(-1,1)
# predictions

In [28]:
print(classification_report(test['category'], predictions))
print(accuracy_score(test['category'], predictions))

              precision    recall  f1-score   support

    accident       0.83      0.91      0.87        11
      murder       0.47      0.90      0.62        10
        rape       1.00      0.67      0.80         9
       theft       1.00      0.59      0.74        17

    accuracy                           0.74        47
   macro avg       0.83      0.77      0.76        47
weighted avg       0.85      0.74      0.76        47

0.7446808510638298


In [29]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(vectors_train, topics_train)

RandomForestClassifier()

In [30]:
# Predict with the testing set
topics_pred = classifier.predict(vectors_val)

# ...and measure the accuracy of the results
from sklearn.metrics import classification_report
print(classification_report(topics_val, topics_pred))

              precision    recall  f1-score   support

    accident       0.97      0.90      0.93        31
      murder       0.76      0.91      0.83        32
        rape       0.95      0.88      0.91        42
       theft       0.91      0.89      0.90        36

    accuracy                           0.89       141
   macro avg       0.90      0.89      0.89       141
weighted avg       0.90      0.89      0.90       141



In [31]:
newcomplaints = [
    "I was a victim of theft. My wallet got stolen.",
    "I witnessed a car accident on Main Street.",
    "My sister has been killed .",
    "my package worth 50$ was stolen.",
    "There has been a murder in my neighborhood. Please investigate.",
    "I suspect my coworker of stealing office supplies.",
]

newvectors = matrix.transform(newcomplaints).toarray()

In [32]:
predictions= [classifier.predict(newvectors)]
predictions

[array(['theft', 'murder', 'murder', 'murder', 'murder', 'murder'],
       dtype=object)]

In [33]:
#test on test data
# test= pd.read_csv('testdata.csv')
complaints= test['complaint']
newvectors = matrix.transform(complaints).toarray()
predictions= classifier.predict(newvectors)
# predictions= predictions.reshape(-1,1)
# predictions

In [34]:
print(classification_report(test['category'], predictions))
print(accuracy_score(test['category'], predictions))

              precision    recall  f1-score   support

    accident       0.62      0.91      0.74        11
      murder       0.45      0.90      0.60        10
        rape       1.00      0.44      0.62         9
       theft       1.00      0.41      0.58        17

    accuracy                           0.64        47
   macro avg       0.77      0.67      0.63        47
weighted avg       0.80      0.64      0.63        47

0.6382978723404256
